In [1]:
import pandas as pd 
import numpy as np 

import psycopg2 as pg2
import sqlalchemy
import s3fs

import julian
import datetime
import calendar
import time

import gdal
import os
import csv

import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib inline

import src.humble_beginnings as helper

In [2]:
metadata,idi=helper.load_metadata(meta_data_path='/Users/Berzyy/projects/plant_forecast/preloaded_data/ghcnd-stations.txt')

In [168]:
station_id='US1COEP0076'

In [117]:
colorado = tuple(metadata[metadata.state=='CO'].station_id.values)

In [118]:
colorado

('US10RMHS145',
 'US1COAD0001',
 'US1COAD0002',
 'US1COAD0003',
 'US1COAD0004',
 'US1COAD0011',
 'US1COAD0012',
 'US1COAD0015',
 'US1COAD0020',
 'US1COAD0022',
 'US1COAD0023',
 'US1COAD0028',
 'US1COAD0029',
 'US1COAD0030',
 'US1COAD0036',
 'US1COAD0039',
 'US1COAD0040',
 'US1COAD0042',
 'US1COAD0044',
 'US1COAD0049',
 'US1COAD0050',
 'US1COAD0051',
 'US1COAD0052',
 'US1COAD0053',
 'US1COAD0059',
 'US1COAD0066',
 'US1COAD0067',
 'US1COAD0069',
 'US1COAD0075',
 'US1COAD0076',
 'US1COAD0082',
 'US1COAD0083',
 'US1COAD0087',
 'US1COAD0091',
 'US1COAD0093',
 'US1COAD0094',
 'US1COAD0096',
 'US1COAD0098',
 'US1COAD0100',
 'US1COAD0103',
 'US1COAD0104',
 'US1COAD0105',
 'US1COAD0107',
 'US1COAD0108',
 'US1COAD0110',
 'US1COAD0117',
 'US1COAD0118',
 'US1COAD0119',
 'US1COAD0120',
 'US1COAD0123',
 'US1COAD0124',
 'US1COAD0126',
 'US1COAD0127',
 'US1COAD0129',
 'US1COAD0130',
 'US1COAD0132',
 'US1COAD0133',
 'US1COAD0134',
 'US1COAD0135',
 'US1COAD0138',
 'US1COAD0139',
 'US1COAD0141',
 'US1COA

In [169]:
def get_one_station(station_id,state_list):
    """Takes in one station and gets the average information
    per month. 
    """
    conn = pg2.connect(dbname='weather', host='localhost')
    cur = conn.cursor()
    conn.autocommit = True

    get_weather_command = f"""SELECT * FROM w_00 
                            WHERE station_id = '{station_id}';"""

    cur.execute(get_weather_command)
    station_data = cur.fetchall()
    station_df = helper.prepare_weather_data_for_merge(station_data)
    pivoted_df = helper.pivot_weather_data_frame(station_df)
    conn.close()
    return pivoted_df
    

In [170]:
raw = get_one_station(station_id,colorado)

In [171]:
def average_by_month(station_df):
    """Takes in a dataframe, averages everythig by month:
    """
    df = station_df.reset_index(level=0)
    month_groups =  df.groupby(by=[df.index.month])
    return month_groups.mean()

In [172]:
average_by_month(raw)

,station_id
measurement_type,
measurement_date,
